In [1]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from random import sample
from datetime import datetime

!pip install -q tensorflow_addons
import tensorflow_addons as tfa

!pip install mne
import mne

!pip install wfdb pyEDFlib PyWavelets eeglib wfdb pyeeg

%matplotlib inline
import wfdb
import pyedflib
import pywt 
import eeglib

from mne import Epochs, create_info, events_from_annotations
from mne.io import concatenate_raws, read_raw_edf
from mne.decoding import CSP
from mne.time_frequency import AverageTFR
from mne.decoding import Vectorizer
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     |████████████████████████████████| 1.7 MB 597 kB/s 
     |████████████████████████████████| 133 kB 63.2 MB/s 
     |████████████████████████████████| 195 kB 50.9 MB/s 
     |████████████████████████████████| 5.8 MB 22.7 MB/s 
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp37-cp37m-linux_x86_64.whl size=382844 sha256=53ca295cade74bbb2ee306ae6c2c46b8c07729ae558dbac5790445bcd0c07094
  Stored in directory: /root/.cache/pip/wheels/a2/da/8c/2d1b9b233595056f05e59156bb555f7277b88beb385605de16
  Created wheel for websocket: filename=websocket-0.2.1-py3-none-any.whl size=192132 sha256=8102fd1e2809789c773a2054eed1f2acf6425f7c9e661337a25d76f989604f10
  Stored in directory: /root/.cache/pip/wheels/e2/9b/2b/6324f465ec5543e33a31c2692c01c1760173bbb98b37dbaee8
Successfully built fastdtw websocket
/kaggle/input/eeg-edfs-files-curated/seizures summary.csv
/kaggle/input/eeg-edfs-files-curated/edfs/chb17a_03.edf
/kaggle/input/eeg-edfs-files-curated/edfs/chb04_08.edf
/kaggle/input/eeg-edfs-files

Next cell has a list of non-conforming EDF files (files where the seizure(s) are either more than one or too close to the beginning or the end of the file, which make them non canditates to homogeneization.
After the list, function `lee_raw()` is defined which reads the selected (random) EDF for each patient (also randomly chosen), omitting the first 60 seconds of the file and applying base filters (60 Hz. noise) and (0-30 Hz. filter for Alpha and Beta sub bands).

In [2]:
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

lista_erroneos = ['chb10_31.edf', 'chb04_05.edf','chb04_08.edf', 'chb09_19.edf', 'chb09_06.edf', 'chb06_18.edf','chb10_20.edf',
                  'chb10_38.edf', 'chb10_12.edf', 'chb23_06.edf', 'chb06_10.edf', 'chb06_24.edf', 'chb06_09.edf', 'chb07_12.edf', 
                  'chb10_27.edf']

df_seizures = pd.read_csv('/kaggle/input/eeg-edfs-files-curated/seizures summary.csv',delimiter=';', encoding='utf8')

df_seizures = df_seizures[~df_seizures['nombre archivo'].isin(lista_erroneos)]

def lee_raw(paciente):
    
    #elegidos=[]

    #for i in np.random.choice(df_seizures['paciente'].unique(), 1):
    #for i in paciente:
        #np.random.choice(df_seizures['paciente'].unique(), 1):
        #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == i]['nombre archivo']))
    #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'],1) )
    
    #elegidos.append(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0])
    
    elegido = df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0]

    # Path with the recordings
    myPath = '/kaggle/input/eeg-edfs-files-curated/edfs/'
    # Ficheros totales (All patients)
    data = os.listdir(myPath)
    
    #epochs_all llevará todos los epochs de todos los pacientes que seleccionemos.
    epochs_all = np.empty(shape=())  # Empty array with size =  1
    data_all = np.empty(shape=()) # Empty array with size =  1
    epochs_all_10 = np.empty(shape=())  # Empty array with size =  1

    count = 0
    
    #print("Elegidos : {}".format(elegidos))
    #for i in elegidos:
        #print("type de i:{} and {}".format(type(i), i))
        #dataPath = os.path.join(myPath, data[i])
    
    dataPath = os.path.join(myPath, str(elegido))
        
    # We read our EDF recording and build a generic 'mapping' of channels that
    # we will remove since they are wrong or reference channels.
    mapping = {'EOG horizontal': 'eog', 'Resp oro-nasal': 'misc',
               'EMG submental': 'misc', 'Temp rectal': 'misc', 
               'Event marker': 'misc', '-': 'misc', '.': 'misc', 'T8-P8':'T8-P8',
               'FC1-Ref':'FC1-Ref', 'FC2-Ref':'FC2-Ref', 'FC5-Ref':'FC5-Ref',
               'FC6-Ref':'FC6-Ref', 'CP1-Ref':'CP1-Ref','CP2-Ref':'CP2-Ref',
               'CP5-Ref':'CP5-Ref', 'CP6-Ref':'CP6-Ref', 'PZ-OZ':'PZ-OZ'}
    
        # We will specifically exclude the channel with issues (T8-P8) and the weird channels (. and -)
    excluded_channels = mapping.keys()
        
    if str(elegido) in df_seizures['nombre archivo'].to_list():
        
        print("Leyendo fichero...:{}".format(elegido))
        print("Desde :{}".format(myPath+elegido))
        count +=1
        raw = mne.io.read_raw_edf(myPath+elegido, exclude=excluded_channels, preload=True, stim_channel='auto', verbose=None)
     
        raw = raw.resample(sfreq=127)

        #raw = mne.io.read_raw_edf(path, exclude=['-', 'T8-P8', '.'], verbose=False, preload=True)

        # Rename EEG channels
        ch_names = {i: i.replace('EEG ', '') for i in raw.ch_names if 'EEG' in i}
        mne.rename_channels(raw.info, ch_names)
        
        
        
        # We then apply notch filter around 60 Hz. (USA) or 50Hz (Europe) This is recommended by some papers to eliminate noise from electrodes (power line).
        raw = raw.notch_filter(freqs=[59.1, 60.9])

        # We then apply the alpha and beta filters that go from 7 to 26/28 Hz (FIRWIN type, default for MNE/Scipy)
        raw = raw.filter(l_freq=8.0, h_freq=29.0)

        # Now we mark the starts and ends of the seizures to obtain the class (Inter, Preictal, Ictal) for each second.
        seizure_init = float(df_seizures[df_seizures['nombre archivo']==elegido]['inicio seizure segundos'].to_list()[0])
        # print("seiz init", seizure_init)
        seizure_end = float(df_seizures[df_seizures['nombre archivo']==elegido]['fin seizure segundos'].to_list()[0])
        duration_seiz = float(seizure_end-seizure_init)
    
        preictal_init = float(seizure_init - 10*60)  # 10 minutos (se puede aumentar)
        preictal_duration = float(seizure_init - preictal_init)
        print("seiz_init, preictal_duration y seizure_end", seizure_init, preictal_duration, seizure_end)

        normal_antes_pre_dur = float(preictal_init)
        normal_despues_seiz_dur = float(3600 - seizure_end)

        this_file_annots = mne.Annotations(onset=[0, preictal_init, seizure_init, seizure_end ],  # in seconds
            duration=[normal_antes_pre_dur, preictal_duration, duration_seiz, normal_despues_seiz_dur],  # in seconds, too
            description=['Inter','Preictal','Ictal', 'Inter'],
            ch_names = None)
        
        raw.set_annotations(this_file_annots)
        
                
        # Recogemos los eventos de las anotaciones (para construir los futuros epochs a partir de eventos)
        # También podríamos usar duration=2.5 y overlap=0.5
        events, events_ids = mne.events_from_annotations(raw, event_id={'Preictal':0, 'Ictal':2, 'Inter':1}, 
                                                         chunk_duration=1, use_rounding=True )
        
        # Finally we crop the first minute to reduce noise (1 m. until patient settles)
        crop_file_mins = 1
        if crop_file_mins > 0:  # Cut start of file
            # Crop raw
            tmin_crop = crop_file_mins * 60
            #tmax = crop_file_mins * 60
            raw.crop(tmin=tmin_crop)
        
        # Save subject and recording information in raw.info
        # Guardamos la info del fichero: paciente/fichero.
        basename = elegido
        #print(int(basename[3:5]))

        subj_nb, rec_nb = str(basename[3:6]), str(basename[7:8])
        # print("Los integers", subj_nb, rec_nb)

        raw.info['patient_info'] = {'id': subj_nb, 'fichero_id': rec_nb}

        print('Patient_info info: {}'.format(raw.info['patient_info'])+'\n')

        if count == 1:

            raw_final = raw.get_data()

        else:

            raw_final = np.append(raw_final, raw.get_data(), axis=1)

        #mne.concatenate_epochs(epochs_list, add_offset=True, *, on_mismatch='raise', verbose=None)

        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

        # Jugando con tmin y tmax podemos obtener epochs de mas o menos duracion y/o solapamiento 
        epochs_all_tmp = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)
        
        # Construimos epochs (con el objeto Raw epochs_all_tmp de cada fichero).
        if data_all.size > 1:

            #epochs_all = np.append(epochs_all, epochs_all_tmp.get_data(), axis=0)

            epochs_PIN = epochs_all_tmp['Preictal', 'Ictal', 'Inter']
            final_df = final_df.append(epochs_PIN.to_data_frame())

            data_all_tmp = epochs_PIN.get_data()
            print('size of tmp : {}'.format(data_all_tmp.shape) +'\n')
            data_all = np.append(data_all, data_all_tmp, axis=0)

            #labels_data_tmp = epochs_all_tmp.events[:,-1]

            labels_data_all = np.append(labels_data_all, epochs_PIN.events[:,-1], axis = 0)

            #epochs_all_10 = np.append(epochs_all_10, epochs_all_tmp_10, axis=0)

        else:
            
            epochs_all = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)

            epochs_PIN = epochs_all['Preictal', 'Ictal', 'Inter']
                
            # Para 'unificar' clases por tamanos. 
            # epochs_PIN.equalize_event_counts(list(events_ids.keys()))
                
            final_df = epochs_PIN.to_data_frame()

            data_all = epochs_PIN.get_data()

            #print('size of dataall :{}'.format(data_all.shape) + '\n')
            labels_data_all = epochs_PIN.events[:,-1]

            #epochs_all_10 = epochs_all_tmp_10

#    else:

#        continue
    final_df['cat'] = [1 if i == 'Preictal' else 2 if i == 'Ictal' else 0 for i in final_df['condition']]
    
    return final_df

In [3]:
def obtain_dwt(DF, wavelet='db4', ventana=10, solapamiento=0, level = 3, fs = 128):
    
    level = level
    fs = fs
    final_df = DF
    wavelet = wavelet
    
    #escalas = pywt.scale2frequency(waveletname.name, np.arange(8,30.5,2)) / (1/fs)
    
    ventana = ventana
    incremento = fs * ventana  # La ventana me da el incremento o 'avance' en cada iteracion
    solapamiento = solapamiento * fs
    
    train_size = int(np.floor(final_df.shape[0]/(ventana*fs)))
    
    n_channels = len(final_df.columns) - 4
    # Redondeamos para evitar el último epoch que desbordaría
    tamano_resultado_train = int(round(train_size,0))
    print("Tamano del loop: {}".format(tamano_resultado_train))
    
    # Calculamos el tamaño del resultado de antemano, para crear la matriz con este tamaño
    #shapes = pywt.wavedecn_shapes((incremento, ), wavelet=wavelet, level=level, axes=(0, ))
    
    #tamano = shapes[0][0] + list(shapes[-1].values())[0][0]
    #print("el tamano", tamano)
    
    # Matriz resultado en train (tamaño muestras, escalas, freq.samp, canales)
    #train_data_dwt = []
    train_data_dwt = np.ndarray(shape=(tamano_resultado_train, fs*ventana , n_channels))
    train_data_dwt_labels = np.ndarray(shape=(tamano_resultado_train, 1, n_channels))
    
    for ii in range(0,tamano_resultado_train):
        if ii == int(np.floor(tamano_resultado_train/2)):
            print("DWT en set training a la mitad...{}".format(ii*incremento))
    
        for jj in range(0,n_channels):
        
            start = final_df['epoch'].min()
 
            signal = final_df.iloc[ii*(incremento - solapamiento):(ii+1)*(incremento-solapamiento), jj+3:jj+4]
            
            #print('Signal', len(signal))
    
            # Calculamos la clase con el 'techo' del valor de la mediana de cada ventana
            clase_intervalo = int(np.ceil(np.median(final_df.iloc[ii*(incremento- solapamiento) : (ii+1)*(incremento- solapamiento), -1])))
  
            train_data_dwt_labels[ii, : ,jj] = clase_intervalo
        
            #coeff, freq = pywt.cwt(signal, escalas, waveletname, 1)
            lista_coeficientes = pywt.wavedec(signal.values.ravel(), wavelet, level=level, mode='smooth')
            tamano = sum([len(i) for i in lista_coeficientes])
            train_data_dwt = np.ndarray(shape=(tamano_resultado_train, tamano , n_channels))
            lista_coeficientes = [abs(item) for sublist in lista_coeficientes for item in sublist]
                        
            #print('Coeff len',  len(lista_coeficientes))
            
            #print('Coeffici len[0]',  len(lista_coeficientes[0]))
            #print('Shape len[-1]',  lista_coeficientes[-1].shape)
            
            #pywt.coeffs_to_array(lista_coeficientes)
            
            #resultados = dict()

            #coeffs = pywt.wavedec([1,2,3,4,5,6,7,8], 'db1', level=nivel)
            #cA2, cD2, cD1 = coeffs

            #for n in range(1, level+1):
            #    if n == level:
                    #print("Nivel", n)
            #        ultimonivel = list(abs(lista_coeficientes[0]))
            #        ultimonivel.extend(list(abs(lista_coeficientes[n])))
            
                    #print(ultimonivel[0].shape)
                    #print("Primero", ultimonivel[0])
                    #print("Ultimo", ultimonivel[-1])
            
            #coeff_approx = coeff[:,:fs]
            #train_data_cwt[ii, :, :, jj] = abs(coeff_approx[:,:,-1])
            
            # Pasamos a ceros los que no nos interesan para que la reconstrucción sea posible, pero con ceros.
            
            #for i in range(level):
            #    if (i > 0 and i < level):
            #        lista_coeficientes[i] = np.zeros_like(lista_coeficientes[i])
            
            # reconstruct = pywt.waverec(lista_coeficientes, wavelet, mode='smooth')
            
            train_data_dwt[ii, :, jj] = lista_coeficientes
            #train_data_dwt[ii, :, jj] = reconstruct
            #train_data_dwt.append((ii, reconstruct, jj))
            
    print("Final DWT decomp. \n") 
    
    return train_data_dwt, train_data_dwt_labels

In [4]:
# Resultados individualizados
segundos = [10, 15, 30, 45]
solapamientos = [0, 0.5]
resultados = dict()
#for i in df_seizures['paciente'].unique():
for i in np.random.choice(df_seizures['paciente'].unique(),12).tolist():
    temp = lee_raw(i)
    for j in segundos:
        resultados.update({'paciente_'+i+'_'+str(j): temp})
        resultados.update({'paciente_'+i+'_'+str(j): obtain_dwt(temp, ventana=j)})

Leyendo fichero...:chb01_26.edf
Desde :/kaggle/input/eeg-edfs-files-curated/edfs/chb01_26.edf
Extracting EDF parameters from /kaggle/input/eeg-edfs-files-curated/edfs/chb01_26.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 595199  =      0.000 ...  2324.996 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 839 samples (6.606 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 29 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband

/tmp/ipykernel_20/4092241418.py:98: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(this_file_annots)


61 bad epochs dropped
Not setting metadata
Not setting metadata
2325 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 2325 events and 128 original time points ...
61 bad epochs dropped
Tamano del loop: 226
DWT en set training a la mitad...144640
Final DWT decomp. 

Tamano del loop: 150
DWT en set training a la mitad...144000
Final DWT decomp. 

Tamano del loop: 75
DWT en set training a la mitad...142080
Final DWT decomp. 

Tamano del loop: 50
DWT en set training a la mitad...144000
Final DWT decomp. 

Leyendo fichero...:chb15_17.edf
Desde :/kaggle/input/eeg-edfs-files-curated/edfs/chb15_17.edf
Extracting EDF parameters from /kaggle/input/eeg-edfs-files-curated/edfs/chb15_17.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Setting up band-stop filter

FIR filter parameters
--------------

/tmp/ipykernel_20/4092241418.py:98: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(this_file_annots)


61 bad epochs dropped
Not setting metadata
Not setting metadata
2274 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 2274 events and 128 original time points ...
61 bad epochs dropped
Tamano del loop: 221
DWT en set training a la mitad...140800
Final DWT decomp. 

Tamano del loop: 147
DWT en set training a la mitad...140160
Final DWT decomp. 

Tamano del loop: 73
DWT en set training a la mitad...138240
Final DWT decomp. 

Tamano del loop: 49
DWT en set training a la mitad...138240
Final DWT decomp. 

Leyendo fichero...:chb02_19.edf
Desde :/kaggle/input/eeg-edfs-files-curated/edfs/chb02_19.edf
Extracting EDF parameters from /kaggle/input/eeg-edfs-files-curated/edfs/chb02_19.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Setting up band-stop filter

FIR filter parameters
--------------

In [5]:
for k, v in resultados.items():
    #['paciente_chb10_10'].values()
    print(v[0].shape, v[1].shape)

(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78, 5780, 21) (78, 1, 21)
(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78, 5780, 21) (78, 1, 21)
(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78, 5780, 21) (78, 1, 21)
(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78, 5780, 21) (78, 1, 21)
(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78, 5780, 21) (78, 1, 21)
(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78, 5780, 21) (78, 1, 21)
(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78, 5780, 21) (78, 1, 21)
(221, 1300, 21) (221, 1, 21)
(147, 1940, 21) (147, 1, 21)
(73, 3860, 21) (73, 1, 21)
(49, 5780, 21) (49, 1, 21)
(353, 1300, 21) (353, 1, 21)
(235, 1940, 21) (235, 1, 21)
(117, 3860, 21) (117, 1, 21)
(78

In [6]:
from tensorflow import keras
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Conv3D, MaxPooling3D
from keras.models import Sequential
from keras.callbacks import History 

#history = History()

def create_model(train_data_dwt, train_data_dwt_labels):
    canales = train_data_dwt.shape[2]
    input_shape = (train_data_dwt.shape[1], train_data_dwt.shape[2])

    num_classes = 3
    batch_size = 32
    epochs = 50

    model = Sequential()
    model.add(Conv1D(32, kernel_size=5, strides=1,
                 activation='relu',
                 input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2, strides=1))
    model.add(Conv1D(64,5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    

# This is the model for the Continuous Case (CWT)
#model.add(Conv2D(32, kernel_size=(5,5), strides=(1,1),
#                 activation='relu',
#                 input_shape=input_shape))
#model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1)))
#model.add(Conv2D(64,(5,5), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Flatten())
#model.add(Dense(1000, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.01),
              metrics= ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
                       ])
    
    return model

In [7]:
from sklearn.preprocessing import StandardScaler
def process_model(data, labels):
    from random import sample
    
    train_data_dwt = data
    train_data_dwt_labels = labels
    
    #print(train_data_cwt.shape)
    #print(train_data_cwt_labels.shape)
    
    items = [item for sublist in train_data_dwt_labels[:,:,-1] for item in sublist]

    import collections

    # using Counter from Dict() to find frequency of elements
    balanceo = collections.Counter(items)
    suma = sum(balanceo.values())
    for k, v in balanceo.items():
        balanceo[k] = (1/v)*(suma/3)
    print("Balanceo clases : {}".format(dict(balanceo)))

    scaler = StandardScaler()
   
    #num_instances, coef, num_canales = train_data_cwt.shape
    train_data = np.reshape(train_data_dwt, (-1, train_data_dwt.shape[-1]))
    train_data = scaler.fit_transform(train_data)
    train_data = np.reshape(train_data, (train_data_dwt.shape[0], train_data_dwt.shape[1], train_data_dwt.shape[-1]))
    
    random_num_generated = int(sample(range(0,99), 1)[0])
    
    # Ensure that we get at least one sample of each class on both train and test
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_dwt_labels,  test_size=0.3, random_state = random_num_generated)
    #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
    y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    #print("Shapes", y_train_cat.shape, y_test_cat.shape)
    
    # Ensure that we get at least one sample of each class on both train and test
    while len(set(np.argmax(y_test_cat, axis=1).tolist())) < len(set(np.argmax(y_train_cat, axis=1).tolist())):
        random_num_generated = int(sample(range(0,99), 1)[0])
        X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_dwt_labels,  test_size=0.3, random_state = random_num_generated)
        y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
        y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    #print("Shapes", y_train_cat.shape, y_test_cat.shape)
    
    batch_size = 32
    epochs = 50
    
    METRICS = ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=len(set(np.argmax(y_test_cat, axis=1).tolist()))),
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
    model = create_model(train_data_dwt, train_data_dwt_labels)
    #model.compile(loss=keras.losses.categorical_crossentropy,
    #          optimizer=keras.optimizers.Adam(learning_rate=0.001),
    #          metrics= METRICS)
    
    history = History()
    history = model.fit(X_train, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_cat),
          #callbacks = create_callbacks(model),
          callbacks = [history],
          class_weight = balanceo)

    ultimos = dict()
    
    for k, v in history.history.items():
        ultimos.update({k: v[-1]})  
    
    return ultimos

Operamos ahora con los resultados de todos los CWTs obtenidos por paciente, ejecutando el modelo CNN para cada uno de los pacientes y guardando los resultados.

In [8]:
historias = dict()
for k, v in resultados.items():
    #a = v[0]
    #b = v[1]
    #print(a.shape, b.shape)
    hist_temp = process_model(v[0], v[1])
    historias.update({k:hist_temp})

Balanceo clases : {0.0: 0.40715109573241065, 1.0: 1.9611111111111112, 2.0: 29.416666666666668}



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Epoch 1/50
8/8 [==============================] - 6s 414ms/step - loss: 2.1742 - accuracy: 0.2308 - tp: 29.0000 - fp: 36.0000 - tn: 458.0000 - fn: 218.0000 - f1_score: 0.1590 - categorical_crossentropy: 1.6783 - precision: 0.4462 - recall: 0.1174 - auc: 0.6022 - prc: 0.4188 - val_loss: 1.0654 - val_accuracy: 0.2170 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 212.0000 - val_fn: 106.0000 - val_f1_score: 0.1189 - val_categorical_crossentropy: 1.0654 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6038 - val_prc: 0.3509
Epoch 2/50
8/8 [==============================] - 2s 235ms/step - loss: 4.0684 - accuracy: 0.3441 - tp: 0.0000e+00 - fp: 1.0000 - tn: 493.0000 - fn: 247.0000 - f1_score: 0.2269 - categorical_crossentropy: 8.2614 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6908 - prc: 0.4654 - val_loss: 0.9708 - val_accuracy: 0.7736 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 212.0000 - val_fn: 106.0000 - val_f1_score: 0.2908 - val_categorical_cros

In [9]:
from datetime import datetime
historias_df = pd.DataFrame(historias)
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
historias_df.to_csv('/kaggle/working/historias_df_DWT_Raw'+dt_string+'.csv', sep=',', na_rep='', header=True, index=True, mode='w')